# PART 2

## Problem （unicode1）: Understanding Unicode （1 point）

```python
>>> chr(0)
'\x00'
>>> print(chr(0))

>>> "this is a test" + chr(0) + "string"
'this is a test\x00string'
>>> print("this is a test" + chr(0) + "string")
this is a teststring
>>>
```

## Problem(unicode2): Unicode Encodings(3 points)

### (a)

- UTF-8 为变长编码，对 ASCII 字符（文本中常见）仅用 1 字节，存储和处理更高效；
- UTF-8 兼容 ASCII，在多语言文本中适应性更强
- 而 UTF-16/32 对 ASCII 文本冗余度高（UTF-16 至少 2 字节，UTF-32 固定 4 字节）。

### (b)

输入 日语或中文即可

### (c)

指的是不符合 UTF-8 规则的


## 2.4 BPE Tokenizer Training



- Vocabulary initialization
- pre-tokenization
- BPE merge
- Special tokens


### Problem: BPE Tokenizer Training (15 points)

这里主要问题在于弄清楚类型都是什么，bytes如何操作

- initial_bytes
  - bytes([i])和 bytes(i)的差异！
  - bytes(i)是i个\0拼起来

- _pre_tokenize
  - 他的任务是，拆分出初步的pretokenize的结果
  - 注意：按照special tokens 拆分开，不能跨越special token进行merge
  - 返回pre_tokens，内容为{tuple(byte1, byte2, ... , bytek) : count}
- train
  - 重点是merge部分的算法，这里是主要时耗
- 为什么采取分离的方式构建vocab?
  - 这样只需要维护`merged`就可以了


- 2025-10-20最新战报：最大堆修正失败，寄
- 2025-10-28 重整旗鼓 over
- 2025-10-29 64G机器还是OOM ToT

- 2025-10-30 打算今天根据一个blog 抄他更改的思路自己实现最后的工业级别的结果

- TODO: `Optionally (this could be a large time-investment),you can implement the key parts of your training method using some systems language, for instanceC++ (considercppyyfor this) or Rust (using PyO3)`

这里是我加速bpe_training的流程思路之类的
> 算法的优化，数据结构的优化，并行(openmp)优化，cython优化，用c++实现关键代码和c++库的cython集成

- 用yield流式读取chunk，一点一点读取
- 为了利用多个CPU进行并行计算，最常见的就是使用多线程。但是由于python的GIL，python的多线程(threading)只能实现并发(concurrency)而不是并行(parallel)，python的多线程只适合I/O密集型任务，而不适合CPU密集型任务。对于CPU密集型任务，比较合适的是使用多进程(multiprocessing)。
- 另外，我们的计算需要分成很多步骤，因此使用队列和生产者/消费者模式来实现是比较清晰的。具体来说我们的流程如下：
  - 文档切割 利用文档分隔特殊字符串<|endoftext|>把原始文本切分成多个chunk，每个chunk包含一个或多个文档。结果放到chunk_queue。
  - 正则匹配和词频统计 把chunk用<|endoftext|>切分成文档，再把文档切分成词，然后统计词频。处理的结果放到counter_queue。
  - 词频合并 从counter_queue取出chunk的词频统计结果，然后进行合并，结果放到merged_queue。
  - 最终合并 主进程把merged_queue里的统计结果进行合并。
- 遇到类似的问题，采用 8-2配置用时40s，采用16-2配置用时30s，采用16-4配置用时 30s


### Problem （train_bpe_tinystories）: BPE Training on TinyStories （2 points）

- 2025-10-29 改了下 pre-tokenize, 防止一下全放到内存里给干炸了
- 跑完了，爽


### Problem （train_bpe_expts_owt）: BPE Training on OpenWeb Text （2 points）
```
 uv run cs336_basics/train_bpe.py
chunk_queu: 0, counter_queue: 0, merged_queue: 0
chunk_queu: 36559, counter_queue: 3, merged_queue: 0
chunk_queu: 67665, counter_queue: 1, merged_queue: 0
chunk_queu: 97751, counter_queue: 1, merged_queue: 0
chunk_queu: 126147, counter_queue: 2, merged_queue: 0
chunk_queu: 153798, counter_queue: 3, merged_queue: 0
chunk_queu: 145248, counter_queue: 0, merged_queue: 0
chunk_queu: 132496, counter_queue: 1, merged_queue: 0
chunk_queu: 119565, counter_queue: 4, merged_queue: 0
chunk_queu: 106843, counter_queue: 4, merged_queue: 0
chunk_queu: 95257, counter_queue: 0, merged_queue: 0
chunk_queu: 82290, counter_queue: 2, merged_queue: 0
chunk_queu: 69311, counter_queue: 3, merged_queue: 0
chunk_queu: 56490, counter_queue: 2, merged_queue: 0
chunk_queu: 43478, counter_queue: 0, merged_queue: 0
chunk_queu: 30519, counter_queue: 0, merged_queue: 0
chunk_queu: 17555, counter_queue: 1, merged_queue: 0
chunk_queu: 4603, counter_queue: 3, merged_queue: 0
chunk_queu: 0, counter_queue: 0, merged_queue: 0
_pretokenize_and_count takes: 190.0569782579987
merge 1000: 231.10991237199778
merge 2000: 371.4132240429972
merge 3000: 575.2559400539976
merge 4000: 836.2143881919983
merge 5000: 1154.4111030869972
merge 6000: 1515.6526871149981
merge 7000: 1920.8361723350026
merge 8000: 2374.731511883001
merge 9000: 2907.8827909429965
merge time: 3333.8703679980026
Training completed in 3552.53 seconds.
Vocab size: 10000
Longest token: b'----------------' (length=16)
```

## 2.6

### Problem （tokenizer）:Implementing the tokenizer （15 points）

- 2025-10-30 差一个 Memory 限制没有过

### Problem （tokenizer_experiments）:Experiments with tokenizers （4 points）

TinyStories 的 tokenizer 在压缩效率（更低的 bytes/token）和处理速度（更高的 MB/s）上均显著优于 OpenWebText 的 tokenizer，且跨数据集时性能依然稳定。这可能与两个数据集的特性有关：TinyStories 多为短文本、简单词汇，其 tokenizer 可能更侧重 “紧凑编码”；而 OpenWebText 文本更复杂（长文本、多样词汇），tokenizer 可能为了覆盖更多复杂模式而牺牲了部分效率。
如果你的需求是高效处理大规模文本（尤其是追求速度和压缩率），TinyStories 的 tokenizer 会是更优选择。


# PART 3

## 3.3

这里在教学 `einops`

## 3.4

本次作业在数学表示上要求使用列向量 ($y = Wx$)，但在实际代码实现中，您应该回归 PyTorch/NumPy 的行向量惯例 ($y = xW^T$) 来利用其内存优化。

- 实现了 linear 和 embedding

## 3.5
> 2025-10-31 有意义的一天
> 可能重要原因之一是在GenAI实现过 MHA和RoPE和GQA

- Root Mean Square Layer Normalization([ ] 为什么work，当时课上也没有说？还是我忘记了)
- SiLU
- SwiGLU
- RoPE
- attention
- MHA

- TransformerBlock
- TransformerLM